# Pre-Processing

In [ ]:
import os
import pathlib
import matplotlib

import numpy as np
import matplotlib.pyplot as plt
import mne
import mne_bids
from mne.preprocessing import (create_eog_epochs,compute_proj_eog)

#matplotlib.use('Agg') #Raises no browsers
matplotlib.use('Qt5Agg') #Raises browsers

In [ ]:
subject = '01'
session = 'WM1'
task = 'isi0'

#bad_list = ['E6', 'E63', 'E64', 'E68', 'E126', 'E127']

#main bad list
bad_list = ['E3', 'E4', 'E5', 'E11', 'E26', 'E27', 'E29', 'E32', 'E33', 'E34', 'E35', 'E39', 'E40', 'E41', 'E45', 'E47', 'E50', 'E54', 'E57', 'E61', 'E68', 'E70', 'E73', 'E76', 'E81', 'E82', 'E89', 'E91', 'E102', 'E103', 'E106', 'E107', 'E109', 'E110', 'E111', 'E116', 'E118', 'E122', 'E123', 'E124', 'E125']


root_path = os.path.join("C:/Users/jacobj/Desktop/VWM_LAB")
bids_root = os.path.join(root_path, 'EGI_BIDS')
output_path = os.path.join(root_path, 'EGI_OUTPUTS')

# Directory

In [ ]:
eeg_path = "eeg"
subname = "sub-"+subject
sesname = "ses-"+session
epoch_reports = "02_epochs"
preprocessed_reports = "03_preprocessed"

In [ ]:
#Makes EEG_OUTPUTS -> subject -> session -> eeg -> preprocessed_reports
if not os.path.exists(os.path.join(output_path, subname, sesname, 
                                   eeg_path, preprocessed_reports)):
    os.makedirs(os.path.join(output_path, subname, sesname, 
                             eeg_path, preprocessed_reports))

In [ ]:
epoch_report_path = os.path.join(root_path, "EGI_OUTPUTS\\sub-"+subject+"\\ses-"+session+"\\eeg\\"+epoch_reports)

In [ ]:
preprocessed_report_path = os.path.join(root_path, "EGI_OUTPUTS\\sub-"+subject+"\\ses-"+session+"\\eeg\\"+preprocessed_reports)

# SSP

In [ ]:
eog_evoked = create_eog_epochs(raw).average(picks='all')
eog_evoked.apply_baseline((None, None))
eog_evoked.plot_joint()

In [ ]:
eog_projs, _ = compute_proj_eog(raw, n_grad=1, n_mag=1, n_eeg=1, reject=None,
                                no_proj=True)

In [ ]:
mne.viz.plot_projs_topomap(eog_projs, info=raw.info)

In [ ]:
fig = mne.viz.plot_projs_joint(eog_projs, eog_evoked, 'eog')
fig.suptitle('EOG projectors')

In [ ]:
eog_projs_bad, _ = compute_proj_eog(
    raw, n_grad=1, n_mag=1, n_eeg=1, reject=None,
    no_proj=True)
fig = mne.viz.plot_projs_joint(eog_projs_bad, eog_evoked, picks_trace='eog')

In [ ]:
evoked_eeg = epochs.average().pick('eeg')
evoked_eeg.del_proj().add_proj(ecg_projs).add_proj(eog_projs)
fig, axes = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
for pi, proj in enumerate((False, True, 'reconstruct')):
    ax = axes[pi]
    evoked_eeg.plot(proj=proj, axes=ax, spatial_colors=True)
    parts = ax.get_title().split('(')
    ylabel = (f'{parts[0]} ({ax.get_ylabel()})\n{parts[1].replace(")", "")}'
              if pi == 0 else '')
    ax.set(ylabel=ylabel, title=f'proj={proj}')
    ax.yaxis.set_tick_params(labelbottom=True)
    for text in list(ax.texts):
        text.remove()
mne.viz.tight_layout()

# Epoch Comparison

In [ ]:
oldepoch_name = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_epo.fif'

In [ ]:
oldepoch_file = os.path.join(epoch_report_path, oldepoch_name)
old_epochs = mne.read_epochs(oldepoch_file)
old_epochs

In [ ]:
total_epochs_number = len(old_epochs.events)
total_epochs_number

In [ ]:
reject_criteria = dict(eeg=100e-6) # 100 µV, what should our rejection criteria be? 

old_epochs.drop_bad(reject=reject_criteria)

In [ ]:
reject_criteria = dict(eeg=100e-6)

old_epochs.drop_bad(reject=reject_criteria)

In [ ]:
old_epochs_number = len(old_epochs.events)
old_epochs_number

# Run New Epochs 

In [ ]:
filter_raw = reconst_raw.filter(l_freq=0.1, h_freq=40) #should be updated?
filter_raw

In [ ]:
if task == 'isi0':
    tmax = 1.5 #1500 ms after the onset
elif task == 'isi50':
    tmax = 1.550 #1550 ms after the onset
elif task == 'isi250':
    tmax = 1.750 #1500 ms after the onset
elif task == 'isi500':
    tmax = 2.0 #2000 ms after the onset
elif task == 'isi750':
    tmax = 2.25 #2250 ms after the onset
elif task == 'isi1000':
    tmax = 2.5 #2500 ms after the onset
elif task == 'isi1250':
    tmax = 2.75 #2750 ms after the onset
elif task == 'isi1500':
    tmax = 3.0 #3000 ms after the onset

In [ ]:
tmin = -0.25 #250 ms before the event

new_epochs = mne.Epochs(reconst_raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    preload=True)
new_epochs

In [ ]:
reject_criteria = dict(eeg=100e-6) # 100 µV, what should our rejection criteria be? 

new_epochs.drop_bad(reject=reject_criteria)

In [ ]:
new_epochs_number = len(new_epochs.events)
new_epochs_number

# Result

In [ ]:
old_epochs.plot(title = 'Old Epochs')
new_epochs.plot(title = 'New Epochs')

In [ ]:
epoch_report_title = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_preprocessed_report.html'
epoch_report = mne.Report(title='Preprocessed Epochs')
epoch_report.add_epochs(epochs=new_epochs, title='Preprocessed Epochs')
epoch_report_path = os.path.join(preprocessed_report_path, epoch_report_title)

epoch_report.save(epoch_report_path, overwrite=True)

In [ ]:
preprocessedname = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_preprocessed_epo.fif'

In [ ]:
new_epoch_path = os.path.join(preprocessed_report_path, preprocessedname)
new_epochs.save(new_epoch_path, overwrite=True)